In [5]:
import requests 
import pandas as pd 
import numpy as np 
import re
import matplotlib.pyplot as plt
import statsmodels.api as sm

### Read in, clean, and aggregate Investor Purchase Data 

In [3]:
df = pd.read_excel("Investor Purchases by Zip Code and Year.xlsx")

In [6]:
#replace zeros with null, so can count total and drop 
df = df.replace({0:np.nan})
#below counts the nulls 
df.Zip.isna().sum()
df["% of Home Purchases by Investors"].isna().sum()
df['% of Low-Priced Homes Purchased by Investors'].isna().sum()
#drop nulls, which will also drop all zero values because of the tranformation above
df = df.dropna()

Count of nulls & zeros in Zip col: 20 for zip, 291 for total % of homes purchease, 
6,287 for low income(~5 percent). Dropping those values in a dataset of 122860 this seems fine 

In [7]:
#filter for cols we want 
df = df[['year', 'Zip', '% of Home Purchases by Investors',
       '% of Low-Priced Homes Purchased by Investors']]
#drop nulls, which will also drop all zero values because of the tranformation above
df = df.dropna()

In [8]:
def add_zero(x):
	"""
	add back leading zeros dropped from zip codes 
	"""

	if len(x['Zip']) == 4:
		return "0" + x['Zip']
	elif len(x['Zip']) == 3:
		return "00" + x['Zip']
	elif len(x['Zip']) == 2:
		return "000" + x['Zip']
	else:
		return x['Zip']

def fix_leading_zeros(df):
	"""
	add back leading zeros dropped from zip codes 
	"""    
	df["Zip"] = df["Zip"].astype(int)
	df["Zip"] = df["Zip"].astype(str)
	df["Zip"] = df.apply(lambda x: add_zero(x), axis=1)
	return df

In [9]:
#fix leading zeros so we can easily match onto census data 
df = fix_leading_zeros(df)

In [10]:
early = range(2005,2010)#2005-2009
late = range(2010,2015)#2010-2014
def group_by_zip(df, timeframe, timeframe_string):
    """
    Input:
        df: Datafarme
        timeframe: list of years as ints 
        timeframe_string: single string of data timeframe 
    Group data by zip, keep only cols we need, create a new column indicating time frame.
    Per instructions from Carolina: "only include zips with non-null values in calculating the average (so for example, 
    if a zip code has 3 years where the value is 0, we’d use 2 years of data to calculate the average)"
    This is occuring becuase we already dropped all the null/zero values and the only values left in the groupby are > 0
    """
    
    df = df[df["year"].isin(timeframe)]
    df = df[['Zip', '% of Home Purchases by Investors',
           '% of Low-Priced Homes Purchased by Investors']] #drop year col before groupby
    #group by zip, create year column
    df = df.groupby("Zip").mean().reset_index()
    df["Time Period"] = timeframe_string
    return df

early_period = group_by_zip(df, early, "2005-2009")
late_period = group_by_zip(df, late, "2010-2014")

### Calling ACS data 

In [15]:
def get_data(year):
    """
    inputs:
        year (str)
    Load data from API to get ACS five. Modify the url with variable names after "get=" 
    to change the items being requested. 
    quins api_key = "bf28a3dbd9697304642d32f7d8551072ee6863ee"
    """
    url = "https://api.census.gov/data/"+year+"/acs/acs5?get=NAME,B02009_001E,B01003_001E,B02001_002E&for=zip%20code%20tabulation%20area:*&key=bf28a3dbd9697304642d32f7d8551072ee6863ee"
    #url = "api.census.gov/data/2014/acs/acs5/subject?get=NAME,group(S2502_C04_001E)&for=zip%20code%20tabulation%20area"
    response = requests.get(url)
    data_1 = pd.DataFrame(response.json())
    return data_1

census_later_demographic = get_data("2014")

In [18]:
#fix header, rename and drop uneeded column 
col_dict = {"B02009_001E":"Black Total Pop","B01003_001E":"Population Total",
            "B02001_002E":"White Population Total","zip code tabulation area":"Zip"}

def make_header(df,col_dict):
    """
    Inputs:
        df
        col_dict: dictionary with mappings of codes to written col nalmes 
    do some basic cleanup of the header row 
    """
    new_header = df.iloc[0] #grab the first row for the header
    df = df[1:] #take the data less the header row
    df.columns = new_header
    df = df.rename(columns=col_dict)
    df = df.drop("NAME",axis=1)
    #df.columns = [codes[x] for x in df.columns]
    return df

census_later_demographic = make_header(census_later_demographic,col_dict)

In [20]:
def make_int(df, col_list):
    """
    Inputs:
        df
        col_list - list of strings of column name to be made into ints so operations can be preformed 
    Make a column an int
    """
    for i in col_list:
        df[i] = df[i].astype(int)
    return df


col_list = ["Black Total Pop","Population Total","White Population Total"]
census_later_demographic = make_int(census_later_demographic, col_list)

In [22]:
#create new columns 
census_later_demographic["Percentage Black Residents"] = census_later_demographic["Black Total Pop"] / census_later_demographic["Population Total"]
census_later_demographic["Percentage POC Residents"] = 1 - ((census_later_demographic["White Population Total"] / census_later_demographic["Population Total"]))
census_later_demographic["Percentage White Residents"] = census_later_demographic["White Population Total"] / census_later_demographic["Population Total"]

In [23]:
#merge late period data on demo 
late_period_demo = census_later_demographic.merge(late_period, on="Zip")

In [24]:
late_period_demo

,Black Total Pop,Population Total,White Population Total,Zip,Percentage Black Residents,Percentage POC Residents,Percentage White Residents,% of Home Purchases by Investors,% of Low-Priced Homes Purchased by Investors,Time Period
0,612,18948,16463,01742,0.032299,0.131148,0.868852,0.097596,0.168119,2010-2014
1,154,14008,13307,01746,0.010994,0.050043,0.949957,0.057023,0.197292,2010-2014
2,185,15604,14418,01748,0.011856,0.076006,0.923994,0.052621,0.153124,2010-2014
3,1034,19464,17652,01749,0.053124,0.093095,0.906905,0.056872,0.112360,2010-2014
4,4439,39141,31222,01752,0.113410,0.202320,0.797680,0.051143,0.066256,2010-2014
...,...,...,...,...,...,...,...,...,...,...
5864,684,29325,15974,92612,0.023325,0.455277,0.544723,0.049128,0.036258,2010-2014
5865,769,25657,14716,92614,0.029972,0.426433,0.573567,0.036719,0.042784,2010-2014
5866,956,21345,11086,92618,0.044788,0.480628,0.519372,0.022842,0.029457,2010-2014
5867,1075,43802,18608,92620,0.024542,0.575179,0.424821,0.029804,0.058352,2010-2014


### Simple Regressions 2010-2014 data

Main takeaways: For every one percent increase in black residents in a neighborhood
there is an associated~.27 percentage increase in the percent of low-priced homes bought by investors.
Slightly lower, but nearly the same effect for all homes, not just low income. 

For all POC the correlation for low income home purchases is still positive, but smaller at ~.134. And as it is the inverse of how POC is calculated, the effect for white populations is ~ -.134.

All effects are statistically significant with a p value of 0. 


In [25]:
y = late_period_demo['% of Low-Priced Homes Purchased by Investors']
X = late_period_demo["Percentage Black Residents"]
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
predictions = model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                         OLS Regression Results                                         
========================================================================================================
Dep. Variable:     % of Low-Priced Homes Purchased by Investors   R-squared:                       0.161
Model:                                                      OLS   Adj. R-squared:                  0.160
Method:                                           Least Squares   F-statistic:                     1122.
Date:                                          Thu, 22 Oct 2020   Prob (F-statistic):          2.78e-225
Time:                                                  12:33:45   Log-Likelihood:                 4411.2
No. Observations:                                          5869   AIC:                            -8818.
Df Residuals:                                              5867   BIC:                            -8805.
Df Model:                                                     1                                         
Covariance Type:                                      nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                          0.1562      0.002     84.330      0.000       0.153       0.160
Percentage Black Residents     0.2780      0.008     33.498      0.000       0.262       0.294
==============================================================================
Omnibus:                     2392.650   Durbin-Watson:                   1.330
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            13417.948
Skew:                           1.878   Prob(JB):                         0.00
Kurtosis:                       9.384   Cond. No.                         5.67
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [59]:
y = late_period_demo['% of Low-Priced Homes Purchased by Investors']
X = late_period_demo["Percentage POC Residents"]
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
predictions = model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                         OLS Regression Results                                         
========================================================================================================
Dep. Variable:     % of Low-Priced Homes Purchased by Investors   R-squared:                       0.050
Model:                                                      OLS   Adj. R-squared:                  0.049
Method:                                           Least Squares   F-statistic:                     306.2
Date:                                          Thu, 22 Oct 2020   Prob (F-statistic):           7.00e-67
Time:                                                  13:41:27   Log-Likelihood:                 4047.0
No. Observations:                                          5869   AIC:                            -8090.
Df Residuals:                                              5867   BIC:                            -8077.
Df Model:                                                     1                                         
Covariance Type:                                      nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                        0.1583      0.003     62.305      0.000       0.153       0.163
Percentage POC Residents     0.1347      0.008     17.500      0.000       0.120       0.150
==============================================================================
Omnibus:                     1918.153   Durbin-Watson:                   1.197
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7611.411
Skew:                           1.580   Prob(JB):                         0.00
Kurtosis:                       7.598   Cond. No.                         5.19
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [53]:
y = late_period_demo['% of Home Purchases by Investors']
X = late_period_demo["Percentage Black Residents"]
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
predictions = model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                   OLS Regression Results                                   
============================================================================================
Dep. Variable:     % of Home Purchases by Investors   R-squared:                       0.478
Model:                                          OLS   Adj. R-squared:                  0.478
Method:                               Least Squares   F-statistic:                     5377.
Date:                              Thu, 22 Oct 2020   Prob (F-statistic):               0.00
Time:                                      12:53:16   Log-Likelihood:                 9106.2
No. Observations:                              5869   AIC:                        -1.821e+04
Df Residuals:                                  5867   BIC:                        -1.820e+04
Df Model:                                         1                                         
Covariance Type:                          nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                          0.0654      0.001     78.559      0.000       0.064       0.067
Percentage Black Residents     0.2735      0.004     73.326      0.000       0.266       0.281
==============================================================================
Omnibus:                     1772.279   Durbin-Watson:                   1.116
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             8639.820
Skew:                           1.377   Prob(JB):                         0.00
Kurtosis:                       8.268   Cond. No.                         5.67
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [58]:
y = late_period_demo['% of Low-Priced Homes Purchased by Investors']
X = late_period_demo["Percentage White Residents"]
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
predictions = model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                         OLS Regression Results                                         
========================================================================================================
Dep. Variable:     % of Low-Priced Homes Purchased by Investors   R-squared:                       0.050
Model:                                                      OLS   Adj. R-squared:                  0.049
Method:                                           Least Squares   F-statistic:                     306.2
Date:                                          Thu, 22 Oct 2020   Prob (F-statistic):           7.00e-67
Time:                                                  13:41:21   Log-Likelihood:                 4047.0
No. Observations:                                          5869   AIC:                            -8090.
Df Residuals:                                              5867   BIC:                            -8077.
Df Model:                                                     1                                         
Covariance Type:                                      nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                          0.2929      0.006     49.426      0.000       0.281       0.305
Percentage White Residents    -0.1347      0.008    -17.500      0.000      -0.150      -0.120
==============================================================================
Omnibus:                     1918.153   Durbin-Watson:                   1.197
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7611.411
Skew:                           1.580   Prob(JB):                         0.00
Kurtosis:                       7.598   Cond. No.                         7.60
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Simple Regressions 2005 – 2009 data
#### Note: using 2010-2014 demo data for 2005-2009 here as earlier zip code level demographic data not available through the API 

Main takeaways: For every one percent increase in black residents in a neighborhood
There is a ~.129 percentage increase in the percent of low-priced homes bought by investors.
Slightly lower, but nearly the same effect for all homes, not just low income. 
These results are statistically significant with a p value of 0.

For all POC the magnitude of the effect is much smaller, with a coefficient of 0.0182. The P value here is still signiticant, but it is 0.009. As above, as the POC figure is the inverse of the white population which has the same P-value and a coefcient of -0.0182.

In [30]:
#merge early period data on demo
early_period_demo = census_later_demographic.merge(early_period, on="Zip")

In [57]:
y = early_period_demo['% of Low-Priced Homes Purchased by Investors']
X = early_period_demo["Percentage Black Residents"]
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
predictions = model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                         OLS Regression Results                                         
========================================================================================================
Dep. Variable:     % of Low-Priced Homes Purchased by Investors   R-squared:                       0.045
Model:                                                      OLS   Adj. R-squared:                  0.045
Method:                                           Least Squares   F-statistic:                     279.3
Date:                                          Thu, 22 Oct 2020   Prob (F-statistic):           2.59e-61
Time:                                                  13:31:43   Log-Likelihood:                 4784.2
No. Observations:                                          5940   AIC:                            -9564.
Df Residuals:                                              5938   BIC:                            -9551.
Df Model:                                                     1                                         
Covariance Type:                                      nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                          0.1219      0.002     69.743      0.000       0.118       0.125
Percentage Black Residents     0.1298      0.008     16.714      0.000       0.115       0.145
==============================================================================
Omnibus:                     3056.385   Durbin-Watson:                   1.187
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            24379.629
Skew:                           2.343   Prob(JB):                         0.00
Kurtosis:                      11.749   Cond. No.                         5.64
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [60]:
y = early_period_demo['% of Low-Priced Homes Purchased by Investors']
X = early_period_demo["Percentage POC Residents"]
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
predictions = model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                         OLS Regression Results                                         
========================================================================================================
Dep. Variable:     % of Low-Priced Homes Purchased by Investors   R-squared:                       0.001
Model:                                                      OLS   Adj. R-squared:                  0.001
Method:                                           Least Squares   F-statistic:                     6.870
Date:                                          Thu, 22 Oct 2020   Prob (F-statistic):            0.00879
Time:                                                  13:43:01   Log-Likelihood:                 4651.1
No. Observations:                                          5940   AIC:                            -9298.
Df Residuals:                                              5938   BIC:                            -9285.
Df Model:                                                     1                                         
Covariance Type:                                      nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                        0.1346      0.002     58.563      0.000       0.130       0.139
Percentage POC Residents     0.0182      0.007      2.621      0.009       0.005       0.032
==============================================================================
Omnibus:                     2809.388   Durbin-Watson:                   1.137
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            18578.108
Skew:                           2.179   Prob(JB):                         0.00
Kurtosis:                      10.488   Cond. No.                         5.17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [49]:
y = early_period_demo['% of Home Purchases by Investors']
X = early_period_demo["Percentage Black Residents"]
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
predictions = model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                   OLS Regression Results                                   
============================================================================================
Dep. Variable:     % of Home Purchases by Investors   R-squared:                       0.237
Model:                                          OLS   Adj. R-squared:                  0.236
Method:                               Least Squares   F-statistic:                     1840.
Date:                              Thu, 22 Oct 2020   Prob (F-statistic):               0.00
Time:                                      12:50:11   Log-Likelihood:                 10643.
No. Observations:                              5940   AIC:                        -2.128e+04
Df Residuals:                                  5938   BIC:                        -2.127e+04
Df Model:                                         1                                         
Covariance Type:                          nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                          0.0553      0.001     84.845      0.000       0.054       0.057
Percentage Black Residents     0.1243      0.003     42.895      0.000       0.119       0.130
==============================================================================
Omnibus:                     1706.561   Durbin-Watson:                   1.103
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6412.181
Skew:                           1.395   Prob(JB):                         0.00
Kurtosis:                       7.258   Cond. No.                         5.64
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [54]:
y = early_period_demo['% of Low-Priced Homes Purchased by Investors']
X = early_period_demo["Percentage White Residents"]
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
predictions = model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                         OLS Regression Results                                         
========================================================================================================
Dep. Variable:     % of Low-Priced Homes Purchased by Investors   R-squared:                       0.001
Model:                                                      OLS   Adj. R-squared:                  0.001
Method:                                           Least Squares   F-statistic:                     6.870
Date:                                          Thu, 22 Oct 2020   Prob (F-statistic):            0.00879
Time:                                                  13:30:29   Log-Likelihood:                 4651.1
No. Observations:                                          5940   AIC:                            -9298.
Df Residuals:                                              5938   BIC:                            -9285.
Df Model:                                                     1                                         
Covariance Type:                                      nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                          0.1528      0.005     28.657      0.000       0.142       0.163
Percentage White Residents    -0.0182      0.007     -2.621      0.009      -0.032      -0.005
==============================================================================
Omnibus:                     2809.388   Durbin-Watson:                   1.137
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            18578.108
Skew:                           2.179   Prob(JB):                         0.00
Kurtosis:                      10.488   Cond. No.                         7.55
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Concat late and early period data sets and write out to csv

In [43]:
fulldata = pd.concat([early_period_demo,late_period_demo])

In [44]:
fulldata.to_csv("Aggregated_Investor_Home_Purchases_by_Zip.csv")